In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import seaborn as sns
# Estilos de Seaborn
sns.set()
sns.set_palette(palette='colorblind')
%matplotlib inline

Armado de set de "entrenamiento"

In [2]:
conversiones = pd.read_pickle("../data/conversiones.pkl")

In [3]:
conversiones.head()

,mes,anio,USER_ID,trimestre
0,1,2018,2483,1
1,1,2018,7088,1
2,1,2018,1787,1
3,1,2018,2085,1
4,1,2018,8270,1


In [5]:
#Me quedo con las columnas USER_ID y trimestre
aux_df=conversiones.loc[:,['USER_ID','trimestre']]

#Me quedo solo con las conversiones del trimestre que me interesa
trimestre_de_interes=1
aux_df=aux_df[aux_df['trimestre'] == trimestre_de_interes]

aux_df.head()

,USER_ID,trimestre
0,2483,1
1,7088,1
2,1787,1
3,2085,1
4,8270,1


In [6]:
#nueva columna indicando que esos users convirtieron
aux_df['convirtio']=True
aux_df.head()

,USER_ID,trimestre,convirtio
0,2483,1,True
1,7088,1,True
2,1787,1,True
3,2085,1,True
4,8270,1,True


In [7]:
aux_df.shape

(277, 3)

In [8]:
#Me deshago de los users repetidos (convirtieron mas de una vez por cuatrimestre)
aux_df.drop_duplicates(['USER_ID'], keep='first', inplace=True)
aux_df.shape

(246, 3)

In [10]:
aux_df=aux_df.loc[:,['USER_ID','convirtio']]
aux_df.head()

,USER_ID,convirtio
0,2483,True
1,7088,True
2,1787,True
3,2085,True
4,8270,True


In [11]:
pageviews=pd.read_pickle('../data/pageviews_total_formateado.pkl')

In [12]:
pageviews.head()

,CONTENT_CATEGORY,CONTENT_CATEGORY_BOTTOM,CONTENT_CATEGORY_TOP,FEC_EVENT,ON_SITE_SEARCH_TERM,PAGE,SITE_ID,USER_ID,mes,trimestre
0,2,2,2,2018-01-01 00:09:17,1,2,2,7237,1,1
1,2,2,2,2018-01-01 00:14:17,1,2,2,4913,1,1
2,2,2,2,2018-01-01 00:14:18,1,2,2,4913,1,1
3,2,2,2,2018-01-01 00:14:26,1,3,3,4913,1,1
4,2,2,2,2018-01-01 00:14:32,1,20,3,4913,1,1


In [13]:
pageviews=pageviews.loc[:,['USER_ID','trimestre']]
pageviews.head()

,USER_ID,trimestre
0,7237,1
1,4913,1
2,4913,1
3,4913,1
4,4913,1


In [14]:
#Me quedo solo con las pageviews del trimestre que me interesa
trimestre_de_interes=1
pageviews=pageviews[pageviews['trimestre'] == trimestre_de_interes]
pageviews.head()

,USER_ID,trimestre
0,7237,1
1,4913,1
2,4913,1
3,4913,1
4,4913,1


In [15]:
#Cuento cuantas visualizaciones tuvo cada user y lo transformo en df
pageviews=pageviews['USER_ID'].value_counts().to_frame()

In [16]:
pageviews['vistas ese cuatrimestre']=pageviews.index
pageviews.reset_index(inplace=True)
pageviews.drop('index', axis=1 , inplace=True)
pageviews.head()

,USER_ID,vistas ese cuatrimestre
0,6528,53
1,5423,7829
2,4819,10712
3,4561,340
4,4546,464


In [19]:
aux_df.head()

,USER_ID,convirtio
0,2483,True
1,7088,True
2,1787,True
3,2085,True
4,8270,True


In [18]:
entrenamiento_df=pageviews.merge(aux_df ,on='USER_ID', how='outer')
entrenamiento_df['convirtio']=entrenamiento_df['convirtio'].fillna(False)
entrenamiento_df.head(10)

,USER_ID,vistas ese cuatrimestre,convirtio
0,6528,53,False
1,5423,7829,False
2,4819,10712,False
3,4561,340,False
4,4546,464,False
5,4100,1496,False
6,4035,36,False
7,3837,8397,False
8,3741,4062,False
9,3648,7873,False


In [20]:
del conversiones
del aux_df
del pageviews

In [21]:
#Quedan muchos usuarios sin conversiones
entrenamiento_df['convirtio'].value_counts()

False    11462
True       418
Name: convirtio, dtype: int64

In [22]:
#204 users no tuvieron vistas en el trimestre 1 pero igual convirtieron en ese trimestre
entrenamiento_df['vistas ese cuatrimestre'].isnull().value_counts()

False    11676
True       204
Name: vistas ese cuatrimestre, dtype: int64

In [23]:
entrenamiento_df['vistas ese cuatrimestre']=entrenamiento_df['vistas ese cuatrimestre'].fillna('0')